In [1]:
!pip install pyspark

Defaulting to user installation because normal site-packages is not writeable


In [9]:
import pyspark

In [10]:
from pyspark.sql import SparkSession

In [11]:
spark = SparkSession.builder.appName('Practice').getOrCreate()

In [68]:
df_spark = spark.read.option('delimiter', ", ").csv('tip1.csv', header=True, inferSchema = True)

In [69]:
df_spark.head(2)

[Row(Name='Sam', Age=28, Experience=4, Salary=82000),
 Row(Name='Sophia', Age=31, Experience=5, Salary=67000)]

In [ ]:
df_spark.printSchema()
df_spark.show()
df_spark.select(['Name', 'Experience']).show()
df_spark.dtypes
df_spark.describe().show()

In [71]:
### adding columns in dataframe
df_spark = df_spark.withColumn('Experience After 2 Years', df_spark['Experience']+2)

In [72]:
df_spark = df_spark.drop('Experience After 2 Years')

In [73]:
### Rename the columns
df_spark.withColumnRenamed('Name', 'FirstName').show()

+---------+---+----------+------+
|FirstName|Age|Experience|Salary|
+---------+---+----------+------+
|      Sam| 28|         4| 82000|
|   Sophia| 31|         5| 67000|
|     John| 26|         2| 45000|
|   Martha| 25|         2| 55000|
|    Emily| 29|         6| 94000|
|  William| 30|         5| 78000|
|  Michael| 23|         1| 41000|
|     Luke| 33|         7| 99000|
|   Olivia| 27|         4| 72000|
|    Grace| 24|         3| 60000|
|    Alice| 28|         4| 83000|
|    Ethan| 32|         6| 92000|
|     Lily| 23|         1| 42000|
|    James| 31|         5| 76000|
|      Mia| 25|         2| 53000|
| Benjamin| 30|         5| 80000|
|      Ava| 34|         7| 98000|
|   Oliver| 29|         4| 71000|
|Charlotte| 22|         1| 43000|
|    Jacob| 27|         3| 62000|
+---------+---+----------+------+
only showing top 20 rows



In [12]:
!pip install gender-guesser

Defaulting to user installation because normal site-packages is not writeable


In [13]:
from gender_guesser.detector import Detector

In [14]:
gender_detector = Detector()

In [15]:
def predict_gender(name):
    gender = gender_detector.get_gender(name)
    if gender in ["unknown", "andy"]:
        return None
    if gender == "mostly_male":
        return "male"
    elif gender == "mostly_female":
        return "female"
    return gender

In [16]:
from pyspark.sql.types import StringType

In [64]:
from pyspark.sql.functions import udf, avg, count, collect_list
predict_gender_udf = udf(predict_gender, StringType())

In [85]:
df_spark_with_gender = df_spark.withColumn("PredictedGender", predict_gender_udf(df_spark["Name"]))

In [103]:
#result = df_spark_with_gender.na.drop(subset="PredictedGender").groupBy("PredictedGender").agg(count('*'), avg("salary").alias("AverageSalary"))
more_common_gender = df_spark_with_gender.dropna().groupBy('PredictedGender').agg(count('*').alias("Occurrences")).first()['PredictedGender']
df_spark_with_gender_filled = df_spark_with_gender.fillna(more_common_gender)

result = df_spark_with_gender_filled.groupBy("PredictedGender").agg(count('*'), avg("salary").alias("AverageSalary"))
result.show()

+---------------+--------+-----------------+
|PredictedGender|count(1)|    AverageSalary|
+---------------+--------+-----------------+
|         female|     502|71782.86852589641|
|           male|     330|70354.54545454546|
+---------------+--------+-----------------+



23/09/24 15:51:26 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 47011315 ms exceeds timeout 120000 ms
23/09/24 15:51:26 WARN SparkContext: Killing executors is not supported by current scheduler.
23/09/24 15:51:27 WARN Executor: Issue communicating with driver in heartbeater
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:322)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:101)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:85)
	at org.apache.spark.storage.BlockManagerMaster.registerBlockManager(BlockManagerMaster.scala:80)
	at org.apache.spark.storage.BlockManager.reregister(BlockManager.scala:641)
	at org.apache.spark.executor.Executor.reportHeartBeat(Executor.scala:1111)
	at org.apache.spark.executor.Executor.$anonfun$heartbeater$1(Executor.scala:244)
	at 

In [183]:
tip2_df = spark.read.option('delimiter', ",").csv('tip2.csv', header=True, inferSchema = True)

In [184]:
tip2_df = tip2_df.withColumn('Gender', predict_gender_udf(tip2_df['Name']))

In [185]:
tip2_df.show()

+-------+----+----------+------+------+
|   Name| Age|Experience|Salary|Gender|
+-------+----+----------+------+------+
|   John|  28|      null| 60000|  male|
|  Alice|null|         4| 48000|female|
|  Ethan|  32|         6| 92000|  male|
|   Lily|  23|         1|  null|female|
|Michael|  22|      null| 41000|  male|
| Sophia|  28|         4| 83000|female|
| Olivia|null|         7| 95000|female|
|   null|  26|         3| 66000|  null|
|  Lucas|  29|         5|  null|  male|
|  Grace|  24|         2|  null|female|
+-------+----+----------+------+------+



In [186]:
type(tip2_df.columns)

list

In [187]:
from pyspark.ml.feature import Imputer
mean_cols = ['Age', 'Experience']
mean_imputer = Imputer()
mean_imputer.setInputCols(mean_cols)
mean_imputer.setOutputCols(["{}_imputed".format(c) for c in mean_cols])
mean_imputer.setStrategy('mean')

Imputer_9ed2390969d9

In [188]:
median_imputer = Imputer()
median_imputer.setInputCol('Salary')
median_imputer.setOutputCol('Salary_imputed')
median_imputer.setStrategy('median')

Imputer_80f86e811af6

In [189]:
tip2_df = mean_imputer.fit(tip2_df).transform(tip2_df)
tip2_df = median_imputer.fit(tip2_df).transform(tip2_df)

In [190]:
mode_gender = tip2_df.groupBy('Gender').agg(count('*').alias('Count')).orderBy('Count', ascending = False).first()[0]
tip2_df = tip2_df.fillna(mode_gender, subset=['Gender'])
tip2_df.show()

+-------+----+----------+------+------+-----------+------------------+--------------+
|   Name| Age|Experience|Salary|Gender|Age_imputed|Experience_imputed|Salary_imputed|
+-------+----+----------+------+------+-----------+------------------+--------------+
|   John|  28|      null| 60000|  male|         28|                 4|         60000|
|  Alice|null|         4| 48000|female|         26|                 4|         48000|
|  Ethan|  32|         6| 92000|  male|         32|                 6|         92000|
|   Lily|  23|         1|  null|female|         23|                 1|         66000|
|Michael|  22|      null| 41000|  male|         22|                 4|         41000|
| Sophia|  28|         4| 83000|female|         28|                 4|         83000|
| Olivia|null|         7| 95000|female|         26|                 7|         95000|
|   null|  26|         3| 66000|female|         26|                 3|         66000|
|  Lucas|  29|         5|  null|  male|         29|   

In [191]:
most_common_name_for_gender_udf = udf(lambda column: type(column), StringType())

In [192]:
tip2_df.show()

+-------+----+----------+------+------+-----------+------------------+--------------+
|   Name| Age|Experience|Salary|Gender|Age_imputed|Experience_imputed|Salary_imputed|
+-------+----+----------+------+------+-----------+------------------+--------------+
|   John|  28|      null| 60000|  male|         28|                 4|         60000|
|  Alice|null|         4| 48000|female|         26|                 4|         48000|
|  Ethan|  32|         6| 92000|  male|         32|                 6|         92000|
|   Lily|  23|         1|  null|female|         23|                 1|         66000|
|Michael|  22|      null| 41000|  male|         22|                 4|         41000|
| Sophia|  28|         4| 83000|female|         28|                 4|         83000|
| Olivia|null|         7| 95000|female|         26|                 7|         95000|
|   null|  26|         3| 66000|female|         26|                 3|         66000|
|  Lucas|  29|         5|  null|  male|         29|   

In [193]:
def mode(l):
    d = {}
    for i in l:
        if d.get(i) == None:
            d[i] = 1
        else:
            d[i] += 1
    return max(d, key=d.get)

In [194]:
mode_udf = udf(mode, StringType())

In [195]:
from pyspark.sql.functions import col, struct
mode_name = tip2_df.groupBy(['Gender']).agg(mode_udf(collect_list('Name')).alias('Names'))
mode_name.show()
gender_names_dict = mode_name.select("Gender", "Names").rdd.collectAsMap()

+------+-----+
|Gender|Names|
+------+-----+
|female|Alice|
|  male| John|
+------+-----+



In [196]:
from pyspark.sql.functions import when
tip2_df.withColumn("Name", when(tip2_df['Gender'] == 'female', gender_names_dict['female']).otherwise(gender_names_dict['male']))
tip2_df.show()

+-------+----+----------+------+------+-----------+------------------+--------------+
|   Name| Age|Experience|Salary|Gender|Age_imputed|Experience_imputed|Salary_imputed|
+-------+----+----------+------+------+-----------+------------------+--------------+
|   John|  28|      null| 60000|  male|         28|                 4|         60000|
|  Alice|null|         4| 48000|female|         26|                 4|         48000|
|  Ethan|  32|         6| 92000|  male|         32|                 6|         92000|
|   Lily|  23|         1|  null|female|         23|                 1|         66000|
|Michael|  22|      null| 41000|  male|         22|                 4|         41000|
| Sophia|  28|         4| 83000|female|         28|                 4|         83000|
| Olivia|null|         7| 95000|female|         26|                 7|         95000|
|   null|  26|         3| 66000|female|         26|                 3|         66000|
|  Lucas|  29|         5|  null|  male|         29|   

In [197]:
tip2_df = tip2_df.drop('Age', 'Experience', 'Salary')
tip2_df.show()

+-------+------+-----------+------------------+--------------+
|   Name|Gender|Age_imputed|Experience_imputed|Salary_imputed|
+-------+------+-----------+------------------+--------------+
|   John|  male|         28|                 4|         60000|
|  Alice|female|         26|                 4|         48000|
|  Ethan|  male|         32|                 6|         92000|
|   Lily|female|         23|                 1|         66000|
|Michael|  male|         22|                 4|         41000|
| Sophia|female|         28|                 4|         83000|
| Olivia|female|         26|                 7|         95000|
|   null|female|         26|                 3|         66000|
|  Lucas|  male|         29|                 5|         66000|
|  Grace|female|         24|                 2|         66000|
+-------+------+-----------+------------------+--------------+



In [201]:
tip2_df = tip2_df.withColumnsRenamed({'Age_imputed': 'Age', 'Experience_imputed': 'Experience', 'Salary_imputed': 'Salary'})

In [202]:
tip2_df.show()
tip2_df.columns

+-------+------+---+----------+------+
|   Name|Gender|Age|Experience|Salary|
+-------+------+---+----------+------+
|   John|  male| 28|         4| 60000|
|  Alice|female| 26|         4| 48000|
|  Ethan|  male| 32|         6| 92000|
|   Lily|female| 23|         1| 66000|
|Michael|  male| 22|         4| 41000|
| Sophia|female| 28|         4| 83000|
| Olivia|female| 26|         7| 95000|
|   null|female| 26|         3| 66000|
|  Lucas|  male| 29|         5| 66000|
|  Grace|female| 24|         2| 66000|
+-------+------+---+----------+------+



['Name', 'Gender', 'Age', 'Experience', 'Salary']